#Week 3 Assignment 1 


Importing important libraries

In [48]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Reading from the link using pd.read_html

In [0]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [0]:
df

We can see that the data is in the form of a list. In order to convert it to a dataframe, we must read the first element.

In [0]:
data=df[0]

Printing first 5 elements of the dataframe

In [54]:
data.head()


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Removing **"Not assigned"** from Borough.

In [20]:
data_clean_borough = data[data['Borough'] != "Not assigned"].reset_index(drop=True)
data_clean_borough.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


Converting "/" to ","

In [21]:
data_clean_borough.replace({"/":","},regex=True,inplace=True)
data_clean_borough.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


Grouping the data by postal code and borough

In [22]:
data_grouped=data_clean_borough.groupby([ "Postal code"	,"Borough"], as_index=False).agg(lambda x: ", ".join(x))
data_grouped.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [23]:
data_grouped.shape

(103, 3)

Replacing "Not assigned" in neighbourhood with Borough

In [0]:
for i in range (data_grouped.shape[0]):
  if data_grouped.iloc[i]['Neighborhood']=="Not assigned":
    data_grouped.iloc[i]['Neighborhood']=data_grouped.iloc[i]['Borough']

In [26]:
data_grouped.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Making a dataframe as shown in the question

In [0]:
list1=["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]
datafinal = pd.DataFrame(columns=data_grouped.columns)
for i in list1:
  datafinal=datafinal.append(data_grouped[data_grouped['Postal code']==i],ignore_index=True)

In [46]:
datafinal

,Postal code,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill , Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford , Maryvale"
7,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower , King and Spadina , Railway Lands , ..."


Printing The shape of dataframe

In [47]:
data_grouped.shape

(103, 3)

Reading location data

In [0]:
df2=pd.read_csv("https://cocl.us/Geospatial_data")

In [56]:
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [57]:
df2.shape

(103, 3)

Dropping Postal Code

In [0]:
df_location=df2.drop(['Postal Code'],axis=1)

In [61]:
df_location.head()

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


Concatinating dataframe with the location dataframe 

In [0]:
dataloc=pd.concat([data_grouped,df_location],axis=1)

In [64]:
dataloc.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Making a dataframe as shown in the question

In [0]:
datafinal2 = pd.DataFrame(columns=dataloc.columns)
for i in list1:
  datafinal2=datafinal2.append(dataloc[dataloc['Postal code']==i],ignore_index=True)

In [67]:
datafinal2.shape

(12, 5)

In [68]:
datafinal2

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford , Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower , King and Spadina , Railway Lands , ...",43.628947,-79.394420
